In [1]:
from chroma_handler import *
import os
import json


input_path = r'S:\SID\Analytics\Working Files\Individual\Florian\Projects\semantic_search\Database\Backup'
google_play = 'Google Play\db_embedded.json'
survey = 'Survey\db_embedded.json'
zendesk = 'Zendesk\db_embedded.json'

google_path = os.path.join(input_path, google_play)
survey_path = os.path.join(input_path, survey)
zendesk_path = os.path.join(input_path, zendesk)


collection_name = 'HRC_multi_source'
persist_dir = r'S:\SID\Analytics\Working Files\Individual\Florian\Projects\semantic_search\Database\ChromaDB'  # folder for storing the persistent DB

2025-02-20 15:46:17,868 - INFO - Environment variables loaded.


In [6]:
# Load the data
google_data = prepare_dataframe('google_play', google_path, output_csv_path=None)           # No output path = no CSV is going to be saved

print("Preview of the resulting DataFrame:")
print(google_data.head())
print("\nNumber of rows:", len(google_data))

Preview of the resulting DataFrame:
           userName                                          userImage  score  \
0     Payton Strode  https://play-lh.googleusercontent.com/a/ACg8oc...      5   
1     Payton Strode  https://play-lh.googleusercontent.com/a/ACg8oc...      5   
2  Rosalie Williams  https://play-lh.googleusercontent.com/a-/ALV-U...      5   
3  Rosalie Williams  https://play-lh.googleusercontent.com/a-/ALV-U...      5   
4     Sandra Canela  https://play-lh.googleusercontent.com/a-/ALV-U...      5   

   thumbsUpCount reviewCreatedVersion replyContent repliedAt appVersion  \
0              0                                                          
1              0                                                          
2              0                                                          
3              0                                                          
4              0                                                          

         longname pp_data_

In [9]:
# Create and populate the database
upsert_chroma_data(google_data, collection_name, persist_dir)

2025-02-20 15:48:02,594 - INFO - Initializing persistent Chroma client via `PersistentClient`.
2025-02-20 15:48:02,610 - INFO - Collection 'HRC_multi_source' exists. Retrieving it.
2025-02-20 15:48:02,610 - INFO - Ensuring 'embedding' column is a list of floats.
2025-02-20 15:48:02,705 - INFO - Deleting any existing items with these 3226 IDs (collection was not newly created).
2025-02-20 15:48:07,228 - INFO - Inserting 3226 rows into 'HRC_multi_source' in batches of 100.
2025-02-20 15:48:07,948 - INFO - Inserted rows 0 to 99.
2025-02-20 15:48:08,547 - INFO - Inserted rows 100 to 199.
2025-02-20 15:48:09,186 - INFO - Inserted rows 200 to 299.
2025-02-20 15:48:09,813 - INFO - Inserted rows 300 to 399.
2025-02-20 15:48:10,359 - INFO - Inserted rows 400 to 499.
2025-02-20 15:48:11,098 - INFO - Inserted rows 500 to 599.
2025-02-20 15:48:11,687 - INFO - Inserted rows 600 to 699.
2025-02-20 15:48:12,323 - INFO - Inserted rows 700 to 799.
2025-02-20 15:48:13,124 - INFO - Inserted rows 800 to 8

In [12]:
# Query the database
# 3) Query the database with some text

query_text = "Breeding of Horses"
# Define a similarity threshold (adjust as needed)
similarity_threshold = 0.54

# Query the database with the similarity threshold instead of top_n
results_df = query_chroma(query_text, collection_name, similarity_threshold=similarity_threshold,
                          persist_path=persist_dir)

if not results_df.empty:
    print("########## Top 5 closest results ##########")
    print(results_df.sort_values(by="distance", ascending=True, inplace=False)[["distance", "document"]].head())
    print("########## Bottom 5 closest results ##########")
    print(results_df.sort_values(by="distance", ascending=True, inplace=False)[["distance", "document"]].tail())

2025-02-20 15:49:28,332 - INFO - Retrieving collection 'HRC_multi_source'.
2025-02-20 15:49:28,332 - WARNING - Number of requested results 5500 is greater than number of elements in index 3226, updating n_results = 3226
2025-02-20 15:49:28,467 - INFO - Received 3226 results. Filtering based on similarity threshold.
2025-02-20 15:49:28,479 - INFO - Filtered results count: 102


########## Top 5 closest results ##########
   distance                                           document
0  0.313729                              Got to breed a horse.
1  0.330359    Learning the secrets to breeding horses is key.
2  0.350750                          Breeding beautiful foals.
3  0.363281                         Breed all types of horses.
4  0.372901  Acquiring the resources for breeding horses is...
########## Bottom 5 closest results ##########
     distance                                           document
97   0.537930    I think it should be easier to get more horses.
98   0.538071  I'm guessing the foal stall needs to be at a h...
99   0.538550    I love this games there is so many horse breed.
100  0.538823  Would also be nice if we could buy stables (no...
101  0.539609      I just wish there would be more horse breeds.


In [11]:
# Query with filter for metadata
query_text = "Breeding of Horses"
# Define a similarity threshold (adjust as needed)
similarity_threshold = 0.54

# Query the database with the similarity threshold instead of top_n
results_df = query_chroma(query_text,
                          collection_name,
                          similarity_threshold=similarity_threshold,
                          persist_path=persist_dir,
                          #where_filters={"pp_data_source": "Flo Play"}                 # Filter for a specific data source
                          )

# if results_df is not empty, display the top 5 and bottom 5 results
if not results_df.empty:
    print("########## Top 5 closest results ##########")
    print(results_df.sort_values(by="distance", ascending=True, inplace=False)[["distance", "document"]].head())
    print("########## Bottom 5 closest results ##########")
    print(results_df.sort_values(by="distance", ascending=True, inplace=False)[["distance", "document"]].tail())

2025-02-20 15:48:57,844 - INFO - Retrieving collection 'HRC_multi_source'.
2025-02-20 15:48:57,860 - WARNING - Number of requested results 5500 is greater than number of elements in index 3226, updating n_results = 3226
2025-02-20 15:48:58,075 - INFO - Received 3226 results. Filtering based on similarity threshold.
2025-02-20 15:48:58,079 - INFO - Filtered results count: 102


########## Top 5 closest results ##########
   distance                                           document
0  0.313729                              Got to breed a horse.
1  0.330359    Learning the secrets to breeding horses is key.
2  0.350750                          Breeding beautiful foals.
3  0.363281                         Breed all types of horses.
4  0.372901  Acquiring the resources for breeding horses is...
########## Bottom 5 closest results ##########
     distance                                           document
97   0.537930    I think it should be easier to get more horses.
98   0.538071  I'm guessing the foal stall needs to be at a h...
99   0.538550    I love this games there is so many horse breed.
100  0.538823  Would also be nice if we could buy stables (no...
101  0.539609      I just wish there would be more horse breeds.


In [10]:
chroma_client = chromadb.PersistentClient(path=persist_dir)
collection = chroma_client.get_collection(name=collection_name)
collection.count()

3226

In [5]:
# delet a chroma collection

chroma_client.delete_collection(collection_name)
